In [1]:
import pandas
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [2]:
#import the data
chads_initial = pandas.read_csv(r'stats\final.csv')

In [3]:
#print total number of games
print(len(chads_initial))

#Select only games where at least half of shots were used for CHAD calculation
chads = chads_initial[chads_initial['Home Shots Used'] >= chads_initial['Home Shots']/2]
chads = chads[chads['Away Shots Used'] >= chads['Away Shots']/2]
chads

632


,Game ID,Home Team,Home Shots,Home Shots Used,Home CHAD,Away Team,Away Shots,Away Shots Used,Away CHAD
0,21500001,ATL,82,82,270.901023,DET,96,95,247.350026
1,21500002,CHI,87,82,267.826323,CLE,94,90,250.778986
2,21500003,GSW,96,96,315.873115,NOP,83,81,283.016460
4,21500005,BOS,85,85,303.649665,PHI,83,83,284.329257
5,21500007,DET,74,74,269.749093,UTA,75,75,256.374995
...,...,...,...,...,...,...,...,...,...
616,21500646,BOS,96,52,233.610996,CHI,89,59,212.122698
618,21500648,NYK,85,83,415.895812,LAC,76,76,408.279779
622,21500652,GSW,90,86,362.180067,IND,95,92,391.541281
623,21500653,LAL,84,83,260.601605,SAS,85,84,272.656325


In [4]:
#create new dataframe containing season average CHAD for each team
away = chads.iloc[:,[5,4]]
away.columns = ['Team', 'CHAD']
home = chads.iloc[:,[1,8]]
home.columns = ['Team', 'CHAD']
achad = pandas.concat([away,home])
achad = achad.groupby('Team').mean()
achad

,CHAD
Team,
ATL,243.196067
BKN,244.904278
BOS,258.254979
CHA,236.771299
CHI,259.065178
CLE,270.630030
DAL,238.977249
DEN,243.400341
DET,253.473370


In [5]:
#add new columns containing season avg CHAD, impact (ie deviation from season avg CHAD), and percent impact
chads = chads.copy()
chads = chads.join(achad, on=['Home Team'])
chads = chads.rename(columns={'CHAD':'Home Avg CHAD'})
chads = chads.join(achad, on=['Away Team'])
chads = chads.rename(columns={'CHAD':'Away Avg CHAD'})
chads['Home Impact'] = chads['Home CHAD'] - chads['Away Avg CHAD']
chads['Away Impact'] = chads['Away CHAD'] - chads['Home Avg CHAD']
chads['Home Impact %'] = 100*chads['Home Impact']/chads['Away Avg CHAD']
chads['Away Impact %'] = 100*chads['Away Impact']/chads['Home Avg CHAD']
chads

,Game ID,Home Team,Home Shots,Home Shots Used,Home CHAD,Away Team,Away Shots,Away Shots Used,Away CHAD,Home Avg CHAD,Away Avg CHAD,Home Impact,Away Impact,Home Impact %,Away Impact %
0,21500001,ATL,82,82,270.901023,DET,96,95,247.350026,243.196067,253.473370,17.427653,4.153960,6.875536,1.708070
1,21500002,CHI,87,82,267.826323,CLE,94,90,250.778986,259.065178,270.630030,-2.803706,-8.286192,-1.035992,-3.198497
2,21500003,GSW,96,96,315.873115,NOP,83,81,283.016460,296.629732,258.803847,57.069267,-13.613272,22.051166,-4.589315
4,21500005,BOS,85,85,303.649665,PHI,83,83,284.329257,258.254979,280.766839,22.882826,26.074277,8.150117,10.096331
5,21500007,DET,74,74,269.749093,UTA,75,75,256.374995,253.473370,258.825111,10.923982,2.901625,4.220604,1.144746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,21500646,BOS,96,52,233.610996,CHI,89,59,212.122698,258.254979,259.065178,-25.454182,-46.132281,-9.825397,-17.863075
618,21500648,NYK,85,83,415.895812,LAC,76,76,408.279779,291.400459,270.723046,145.172766,116.879319,53.624089,40.109518
622,21500652,GSW,90,86,362.180067,IND,95,92,391.541281,296.629732,239.585778,122.594289,94.911549,51.169268,31.996641
623,21500653,LAL,84,83,260.601605,SAS,85,84,272.656325,270.490961,250.272681,10.328923,2.165364,4.127068,0.800531


In [24]:
#create new dataframe containg teams average percent impact over the partial season
away = chads.iloc[:,[5,14]]
away.columns = ['Team', 'Impact']
home = chads.iloc[:,[1,13]]
home.columns = ['Team', 'Impact']
avgImpact = pandas.concat([away,home])
t = avgImpact.copy()
avgImpact = avgImpact.groupby('Team').mean()

#sort the dataframe by impact
avgImpact.sort_values(by='Impact')
avgImpact

t.groupby('Team').count().min()

Impact    15
dtype: int64

In [7]:
#get some season stats from Basketball Reference to compare

#https://www.basketball-reference.com/leagues/NBA_2016.html#all_per_game_team-opponent
#Sports Reference LLC "(title of a particular page or blank for general citation)." Basketball-Reference.com - Basketball Statistics and History. https://www.basketball-reference.com/. (date of your visit)
#Provided by Basketball-Reference.com
#Generated 12/1/2021.
teams = {'Atlanta Hawks':'ATL', 'Boston Celtics':'BOS', 'Brooklyn Nets':'BKN', 'Charlotte Hornets':'CHA', 'Chicago Bulls':'CHI',
        'Cleveland Cavaliers':'CLE', 'Dallas Mavericks':'DAL', 'Denver Nuggets':'DEN','Detroit Pistons':'DET','Golden State Warriors':'GSW',
        'Houston Rockets':'HOU', 'Indiana Pacers':'IND','Los Angeles Clippers':'LAC','Los Angeles Lakers':'LAL', 'Memphis Grizzlies':'MEM',
        'Miami Heat':'MIA','Milwaukee Bucks':'MIL','Minnesota Timberwolves':'MIN','New Orleans Pelicans':'NOP','New York Knicks':'NYK',
        'Oklahoma City Thunder':'OKC','Orlando Magic':'ORL','Philadelphia 76ers':'PHI','Phoenix Suns':'PHX','Portland Trail Blazers':'POR',
        'Sacramento Kings':'SAC','San Antonio Spurs':'SAS','Toronto Raptors':'TOR','Utah Jazz':'UTA','Washington Wizards':'WAS'}

team_stats_per100 = pandas.read_csv(r'stats\15-16 Team Stats.csv')
team_stats_per100 = team_stats_per100.iloc[:,[1,8,9]]
team_stats_per100.sort_values(by='Team')
team_stats_per100['Team'] = team_stats_per100['Team'].str.replace('*','')
team_stats_per100['Team Code'] = team_stats_per100['Team'].map(teams)
team_stats_per100

,Team,3PA,3P%,Team Code
0,Golden State Warriors,31.5,0.416,GSW
1,Oklahoma City Thunder,24.3,0.349,OKC
2,Cleveland Cavaliers,31.5,0.362,CLE
3,San Antonio Spurs,19.7,0.375,SAS
4,Toronto Raptors,25.0,0.370,TOR
5,Portland Trail Blazers,29.5,0.370,POR
6,Houston Rockets,31.4,0.347,HOU
7,Los Angeles Clippers,27.7,0.364,LAC
8,Charlotte Hornets,30.5,0.362,CHA
9,Boston Celtics,26.4,0.335,BOS


In [8]:
#Split impact by conference
western = ['UTA', 'PHX', 'DEN', 'LAC', 'DAL', 'POR', 'LAL', 'MEM', 'GSW', 'SAS', 'NOP', 'SAC', 
           'MIN', 'OKC', 'HOU']
eastern = ['PHI', 'BKN', 'MIL', 'NYK', 'ATL', 'MIA', 'BOS', 'WAS', 'IND', 'CHA',
          'CHI', 'TOR', 'CLE', 'ORL', 'DET']

print(avgImpact[avgImpact.index.isin(western)])

         Impact
Team           
DAL   -8.086820
DEN    1.730872
GSW   21.737073
HOU   -2.045163
LAC    1.546732
LAL    0.544037
MEM  -15.974134
MIN   -6.633374
NOP   -5.543590
OKC   15.598830
PHX    2.519368
POR    2.168628
SAC    8.327226
SAS  -10.786896
UTA   -4.458060


In [9]:
print(avgImpact[avgImpact.index.isin(eastern)])

         Impact
Team           
ATL   -0.412776
BKN   -8.590453
BOS   -1.281988
CHA    0.407925
CHI   -5.248573
CLE    1.529636
DET   -1.863779
IND   -2.056865
MIA   -3.267804
MIL   -9.294658
NYK    9.272102
ORL   -5.585111
PHI   10.853412
TOR   -7.779531
WAS    3.861045


In [10]:
#add percentage of shots used for each game
chads_initial['Home Shots %'] = chads_initial['Home Shots Used']/chads_initial['Home Shots']
chads_initial['Away Shots %'] = chads_initial['Away Shots Used']/chads_initial['Away Shots']

In [11]:
chads_initial

,Game ID,Home Team,Home Shots,Home Shots Used,Home CHAD,Away Team,Away Shots,Away Shots Used,Away CHAD,Home Shots %,Away Shots %
0,21500001,ATL,82,82,270.901023,DET,96,95,247.350026,1.000000,0.989583
1,21500002,CHI,87,82,267.826323,CLE,94,90,250.778986,0.942529,0.957447
2,21500003,GSW,96,96,315.873115,NOP,83,81,283.016460,1.000000,0.975904
3,21500004,ORL,100,54,231.557142,WAS,84,32,189.667696,0.540000,0.380952
4,21500005,BOS,85,85,303.649665,PHI,83,83,284.329257,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
627,21500659,CLE,94,0,NaN,CHI,87,0,NaN,0.000000,0.000000
628,21500660,PHX,85,49,225.107884,ATL,84,37,219.650435,0.576471,0.440476
629,21500661,DEN,78,2,416.907815,DET,84,3,386.996662,0.025641,0.035714
630,21500662,POR,93,68,234.474393,LAL,69,57,229.307558,0.731183,0.826087


In [12]:
#total percentage of shots used
(chads_initial['Home Shots %'].mean() + chads_initial['Away Shots %'].mean())/2

0.6449146331249647